In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ==== Import Libraries ====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
# ==== Step 1: Load and Explore the Dataset ====
df = pd.read_csv("/content/drive/MyDrive/Customer Churn Prediction/Churn_Modelling.csv")  # Update with actual file path

print("Dataset Overview:")
print(df.head())

Dataset Overview:
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63      

In [4]:

# ==== Step 2: Drop Irrelevant Columns ====
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])  # These are not useful for churn prediction

In [5]:
# ==== Step 3: Handle Missing Values ====
# Identify numeric and categorical columns
numeric_cols = df.select_dtypes(include=["number"]).columns
categorical_cols = df.select_dtypes(include=["object"]).columns

# Define transformations: Median imputation for numeric, Most frequent for categorical
imputer_numeric = SimpleImputer(strategy="median")
imputer_categorical = SimpleImputer(strategy="most_frequent")

df[numeric_cols] = imputer_numeric.fit_transform(df[numeric_cols])
df[categorical_cols] = imputer_categorical.fit_transform(df[categorical_cols])


In [6]:
# ==== Step 4: Encode Categorical Features (One-Hot Encoding) ====
df = pd.get_dummies(df, columns=["Geography", "Gender"], drop_first=True)

In [7]:
# ==== Step 5: Define Features and Target ====
X = df.drop(columns=["Exited"])  # Features
y = df["Exited"]  # Target variable (1 = Churned, 0 = Not Churned)

In [8]:
# ==== Step 6: Train-Test Split ====
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:

# ==== Step 7: Feature Scaling ====
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# ==== Step 8: Train Machine Learning Models ====
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"\n{name} Model Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))


Logistic Regression Model Accuracy: 0.8080
              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89      1593
         1.0       0.59      0.19      0.28       407

    accuracy                           0.81      2000
   macro avg       0.71      0.58      0.59      2000
weighted avg       0.78      0.81      0.77      2000


Random Forest Model Accuracy: 0.8640
              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92      1593
         1.0       0.78      0.46      0.58       407

    accuracy                           0.86      2000
   macro avg       0.83      0.71      0.75      2000
weighted avg       0.86      0.86      0.85      2000


XGBoost Model Accuracy: 0.8530
              precision    recall  f1-score   support

         0.0       0.88      0.95      0.91      1593
         1.0       0.70      0.49      0.58       407

    accuracy                           0.85      2000
   macro avg  

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:49:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [13]:
# ==== Step 9: Feature Importance Analysis ====
plt.figure(figsize=(10, 5))

# Feature Importance from XGBoost
xgb_model = models["XGBoost"]
feature_importances = pd.Series(xgb_model.feature_importances_, index=X.columns)
feature_importances.sort_values(ascending=False).plot(kind="bar", title="Feature Importance (XGBoost)")

plt.show()